# Classes

Comme on l'a vu, en python, tout est objet.

Les classes sont incontournables. 

Elles permettent de structurer "proprement" un projet, en encapsulant des valeurs et des fonctionnalités au sein d'une même entité.

## Exemple

In [ ]:
class Counter:
    def reset(self,init=0):
        self.count = init
    def add(self):
        self.count += 1
        return self.count
    
counter = Counter()
counter.reset(0) 
print(counter.add())
print(counter.add())

Commentaires :
- Mot clef `class`
- Les methodes sont des fonctions dont le premier argument est `self`
- `self` represente l'objet, instance de la classe
- `self` a des attributs, accessibles par l'opérateur `.` ils peuvent être ajoutés et modifiés **sans contraintes**
- Une classe est instanciée par son appel (`counter = Counter()`)

Les attributs peuvent être aussi modifiés et ajoutés depuis l'objet lui même !

In [ ]:
obj1 = Counter()
obj1.x = 5
print("x in obj1 =", obj1.x)

### Initialisation

In [ ]:
class Counter:
    
    def __init__(self, step=1):
        self.step = step
        self.count = 0
    
    def reset(self,init=0):
        self.count = init
        
    def add(self):
        self.count += self.step
        return self.count
    

`__init__`, qui se prononce "dunder init" est apellé au moment de l'instanciation. 

In [ ]:
c = Counter(step=2)
print(c.add())
print(c.add())
print(c.add())

**dir()** est utile pour connaître les attributs et méthodes accessibles

In [ ]:
print("Contents of Counter class:",dir(Counter) )
print("------------------------")
print("------------------------")
print("Contents of counter object:", dir(c))

## `__dict__`

`__dict__` est le dictionnaire des attributs de l'object. 

On peut le consulter ou le manipuler directement.

In [ ]:
print(c.__dict__)
c.newattr = "newval"
print(c.__dict__)
print(c.__dict__["newattr"])
c.__dict__["newattr"] = "another val"
print(c.newattr)
print(c.__dict__)

On voit ici que les attributs d'un object sont implémentés comme un dictionnaire python. Cela a des coûts en taille et en performance. 

Si l'ensemble des attributs possible est fixé d'avance, il est possible d'empecher python d'utiliser un dictionnaire, par l'utilisation de `__slots__`.

In [ ]:
class Counter:
    __slots__= ("step", "count") 
    def __init__(self, step=1):
        self.step = step
        self.count = 0
    
    def reset(self,init=0):
        self.count = init
        
    def add(self):
        self.count += self.step
        return self.count

In [ ]:
Counter().__dict__   # AttributeError : 'Counter' object has no attribute '__dict__'
Counter().newattr = "newval"  # AttributeError: 'Counter' object has no attribute 'newattr'

Plus d'informations [ici](https://stackoverflow.com/questions/472000/usage-of-slots)

Nous reconnaissons les attributs et méthodes que nous avons défini (add, count, reset, step, `__init__`) mais aussi un certain nombres de méthodes et attributs herités du type `Object`. 

Ces méthodes sont utilisés selon le contexte d'utilisation de l'objet. Quelques exemples :

- `__hash__()` invoqué en interne pour obtenir un hash de l'objet (utile dans le cadre de sets ou de dictionnaires)
- `__getattribute__()` invoqué lorsqu'on recherche un attribut.
- `__str__()` invoqué lors de str(monobjet) ou print(monobjet)
- `__repr__()` utilisé à la place de `__str__()` si ce dernier n'est pas défini. Vise à fournir sous forme de texte une représentation plus précise que `__str__()`.

Il est possible, et parfois utile, de les redéfinir soi même (`__init__` par exemple).

Beaucoup d'autres dunders methodes (autre que celles déjà présentes) peuvent être définis pour augmenter les fonctionnalités de la classe :

- `__getattr__` invoqué lorsqu'un attribut est introuvable
- `__iter__` invoqué lorsqu'on veut itérer sur votre object (comme dans le cadre d'une boucle `for`)
- `__len__` invoqué lors de `len(monobjet)`
- `__contains__` invoqué lors de `"x" in monobjet`
- `__le__`, `__eq__` ... invoqués lors de `monobjet > y`
- `__add__`, `__iadd__` ... invoqués lors de `monobjet + y` ; `monobjet+=y`; tous les opérateurs ont leurs dunders
- `__bytes__`, invoqué lors de `bytes(monobjet)`
- `__bool__`, invoqués lors de `if monobjet`
- `__getitem__`, invoqués lors de `monobjet["x"]`
...

Les dunders ne doivent pas être considérés comme des attributs ou méthodes privées. Mais plutôt comme des méthodes utiles (on les apelle d'ailleurs parfois "magic methods").


Le sujet des dunder methods est riche et vaste. Liens pour en savoir plus :
- [Documentation](https://docs.python.org/3/reference/datamodel.html) . 
- [Excellente vidéo](https://www.youtube.com/watch?v=M4gPxbo6G6k) de l'auteur du livre du livre Fluent Python, portant notamment sur l'utilité des dunders.

## Heritage

Python suporte l'héritage

In [ ]:
class Person:
    newatr = "newval"
    def __init__(self, name):
        self.name = name 
    
    def say_hello(self):
        print("hello")

In [ ]:
class Developper(Person):
    
    def code(self):
        print("I code")

In [ ]:
tom = Developper("tom")
tom.code()
tom.say_hello()
print(tom.name)

## Heritage multiple

L'héritage multiple est possible

In [ ]:
class YiellingAbility:
    def yell(self):
        print("AHHHHHHHHH")
 
class Developper(Person, YiellingAbility):
    def code(self):
        print("I code")

In [ ]:
tom = Developper("tom")
tom.yell()  

## Descriptors

Résumé grossier (https://docs.python.org/3.7/howto/descriptor.html pour en savoir plus)

On peut controller ce qu'il se passe lorsqu'on utilise `object.attribute` dans differents contextes :
- accès
- suppression
- modification

**Exemple** (source : https://docs.python.org/3.7/howto/descriptor.html )

In [ ]:
class RevealAccess:
    """A data descriptor that sets and returns values
       normally and prints a message logging their access.
    """

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name)
        return self.val

    def __set__(self, obj, val):
        print('Updating', self.name)
        self.val = val

class MyClass:
    x = RevealAccess(10, 'var "x"')
    y = 5

In [ ]:
m = MyClass()
print(m.x)
m.x = 20
m.x
m.y

On peut créer soi même des descriptors, ou utiliser des raccourcis pour produire le comportement souhaité. Un raccourci fréquemment utilisé est l'utilisation du decorateur `@property`

In [ ]:
# source : https://www.programiz.com/python-programming/property

class Celsius:
    def __init__(self, temperature = 0):
        self._temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    @property
    def temperature(self):
        print("Getting value")
        return self._temperature

    @temperature.setter
    def temperature(self, value): 
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value

In [ ]:
temp = Celsius(120)

In [ ]:
temp.temperature

In [ ]:
temp.temperature = -275

Résumé sur l'accès aux attributs :
   - Python permet de maitriser très finement ce qui se passe lors de l'accès et la modification d'attributs (a = x.y ou x.y = a)
   - Il y a des couches avant/après l'accès au véritable attribut qu'on peut manipuler
   - Une possibilité est l'utilisation de descripteurs
   - Il est aussi possible d'implémenter `__getattr__` (en aval, si l'attribut rechercé n'est pas trouvé) et/ou `__get_attribute__` (en amont, apellé dans tous les cas).

## Les dunders contrôlant l'accès aux attributs

La recherche d'attribut va de fallbacks en fallbacks : 
   - d'abord dans les attributs de l'objet, puis les attributs de sa classe, puis classe mère. etc...
   - Les dunders get_attr et get_attribute permettent de se brancher à différents endroits dans cette recherche
   - get_attr est apellé **avant** toute recherche
   - get_attribute est apellé **après** la recherche d'attribut dans l'objet (s'il ne le trouve pas) et **avant** la recherche dans les attributs de classes.

En js o.c est équivalent en o["c"]
Pas en python. Ce sont deux méthodes d'accès différentes.

Supposons qu'on aimerait une structure de données plus proche de l'objet javascript. On voudrait une classe qui : 
- S'instancie exactement comme un dictionnaire
- Mais qui permette d'accéder aux clefs par attributs (x.y).

In [ ]:
class Bunch:
    def __init__(self, *args, **kwargs):
        self._dict = dict(*args, **kwargs)
        
    def __getattr__(self, attribute):
        return self._dict[attribute]

In [ ]:
bunch = Bunch(a="a")

In [ ]:
bunch.a

In [ ]:
bunch["a"]

Comment pourrait-on rendre possible

In [ ]:
class Bunch:
    def __init__(self, *args, **kwargs):
        self._dict = dict(*args, **kwargs)
        
    def __getattr__(self, attribute):
        return self._dict[attribute]

    def __getitem__(self, key):
        return self._dict[key]

In [ ]:
bunch = Bunch(a="a")
bunch.a == bunch["a"]

Ou encore

In [ ]:
class Bunch:
    def __init__(self, *args, **kwargs):
        self._dict = dict(*args, **kwargs)
        
    def __getattr__(self, attribute):
        return self._dict[attribute]

    __getitem__=__getattr__